# N-Grams

In [1]:
import logging
from ekorpkit import eKonf

logging.basicConfig(level=logging.WARNING)
print(eKonf.__version__)

0.1.33+3.g88a2e60


## Tokenize and extract tokens

In [2]:
corpus_cfg = eKonf.compose("corpus")
corpus_cfg.name = "bok_minutes"
corpus_cfg.data_dir = "/workspace/data/datasets/corpus/ekorpkit"


In [3]:
tkn_cfg = eKonf.compose("preprocessor/tokenizer=mecab")
cfg = eKonf.compose("pipeline")
cfg.data.corpus = corpus_cfg
cfg._pipeline_ = ["tokenize", "explode_splits", "save_dataframe"]
cfg.num_workers = 100
cfg.tokenize.preprocessor.tokenizer = tkn_cfg
cfg.explode_splits.id_key = "id"
cfg.explode_splits.split_key = "sent_id"
cfg.explode_splits.separator = "\n"
cfg.save_dataframe.output_dir = "../data/bok"
cfg.save_dataframe.output_file = "bok_minutes_tokenized.parquet"

df = eKonf.instantiate(cfg)
df.tail()

Tokenizing column: text:   0%|          | 0/182 [00:00<?, ?it/s]

,id,text,split,sent_id
181,181,또한/MAJ /SP 글로벌/NNG /SP 공급/NNG 망/NNG /SP 재편/NNG...,train,321
181,181,향후/NNG /SP 경제/NNG 회복세/NNG 와/JC /SP 물가/NNG 의/JK...,train,322
181,181,,train,323
181,181,Government/SL ’/SY s/SL /SP View/SL,train,324
181,181,,train,325


In [4]:
tkn_cfg = eKonf.compose("preprocessor/tokenizer=mecab_econ")
tkn_cfg.extract.strip_pos = False

cfg = eKonf.compose("pipeline")
cfg.data.data_dir = "../data/bok"
cfg.data.data_file = "bok_minutes_tokenized.parquet"
cfg._pipeline_ = ["extract_tokens", "save_dataframe"]
cfg.num_workers = 100
cfg.verbose = True
cfg.extract_tokens.preprocessor.tokenizer = tkn_cfg
cfg.extract_tokens.nouns_only = False
cfg.save_dataframe.output_dir = "../data/bok"
cfg.save_dataframe.output_file = "bok_minutes_tokens.parquet"
df = eKonf.instantiate(cfg)
df.tail()

Extracting column: text:   0%|          | 0/36191 [00:00<?, ?it/s]

,id,text,split,sent_id
181,181,또한/MAJ 글로벌/NNG 공급/NNG 망/NNG 재편/NNG 기후/NNG 변화/N...,train,321
181,181,향후/NNG 경제/NNG 회복세/NNG 와/JC 물가/NNG 의/JKG 흐름/NNG...,train,322
181,181,,train,323
181,181,Government/SL s/SL View/SL,train,324
181,181,,train,325


## Train ngrams

In [21]:
ngram_cfg = eKonf.compose("model/ngram=npmi")
ngram_cfg.data.data_dir = "../data/bok"
ngram_cfg.data.data_file = "bok_minutes_tokens.parquet"
ngram_cfg.verbose = True
ngram_cfg.auto.load = True
ngram_cfg.candidates.threshold = 0.4
eKonf.print(ngram_cfg)
# ngram = eKonf.instantiate(ngram_cfg)

{'_method_': ['initialize'],
 '_target_': 'ekorpkit.models.ngram.base.Ngrams',
 'auto': {'load': True},
 'candidates': {'max_candidates': 40000000,
                'min_count': 10,
                'num_sents_per_pruning': 10000,
                'prune_min_ngram_count': 2,
                'threshold': 0.4},
 'data': {'_target_': 'ekorpkit.pipelines.data.Data',
          'column_info': {'_target_': 'ekorpkit.info.column.CorpusInfo',
                          'columns': {'id': 'id',
                                      'merge_meta_on': 'id',
                                      'text': 'text',
                                      'timestamp': None},
                          'data': {'id': 'int', 'text': 'str'},
                          'datetime': {'columns': None,
                                       'format': None,
                                       'rcParams': None},
                          'meta': None,
                          'segment_separator': '\\n\\n',
            

In [6]:
print(len(ngram.ngrams), len(ngram.candidates))

0 41005


In [7]:
ngram.postag_rules = [["/NN", "/J", "/NN"], ["/XP", "/NN"], ["/XP", "/NN", "/NN"]]
# ngram.postag_rules = ['/NN']

ngram.export_ngrams(threshold=0.8, apply_postag_rules=True)

[(('민/XPN', '감도/NNG'),
  score(words='민/XPN;감도/NNG', score=0.9905600642226081, frequency=27, length=44827)),
 (('신/XPN', '흥국/NNG'),
  score(words='신/XPN;흥국/NNG', score=0.9602497345784399, frequency=999, length=44827)),
 (('문안/NNG', '을/JKO', '작성/NNG'),
  score(words='문안/NNG;을/JKO;작성/NNG', score=0.9550163996733796, frequency=43, length=44827)),
 (('예/NNG', '의/JKG', '주시/NNG'),
  score(words='예/NNG;의/JKG;주시/NNG', score=0.952158528049305, frequency=193, length=44827)),
 (('한국/NNP', '은/JX', '행법/NNG'),
  score(words='한국/NNP;은/JX;행법/NNG', score=0.9215496994789585, frequency=13, length=44827)),
 (('한/NNP', '은/JX', '법/NNG'),
  score(words='한/NNP;은/JX;법/NNG', score=0.9115135958462448, frequency=17, length=44827)),
 (('차관/NNG', '에게/JKB', '발언/NNG'),
  score(words='차관/NNG;에게/JKB;발언/NNG', score=0.8730837780556813, frequency=48, length=44827)),
 (('취지/NNG', '로/JKB', '발언/NNG'),
  score(words='취지/NNG;로/JKB;발언/NNG', score=0.8703677143686631, frequency=36, length=44827)),
 (('신/XPN', '용스/NNP', '프레드/NNP'),

In [20]:
ngram.load_data()
sentence = ngram.sentences[18]
print(sentence)
tokens = ngram.ngramize_sentence(sentence, strip_pos=False)
print(tokens)

이/NP 에/JKB 대해/VV+EC 관련/NNG 부서/NNG 에서/JKB 는/JX 실업/NNG 률/XSN 과/JC 경기/NNG 지수/NNG 와/JKB 의/JKG 상관/NNG 관계/NNG 는/JX 그렇게/MAG 높/VA 지/EC 않/VX 은/ETM 것/NNB 으로/JKB 나타나/VV 며/EC 통계청/NNG 에서/JKB 발표/NNG 하/XSV 는/ETM 공식/NNG 적/XSN 인/VCP+ETM 실업/NNG 률/XSN 을/JKO 구직/NNG 단념/NNG 자/XSN 를/JKO 포함/NNG 한/XSA+ETM 광의/NNG 의/JKG 실업/NNG 률/XSN 로/JKB 전환/NNG 할/XSV+ETM 경우/NNG 오히려/MAJ 경기/NNG 와/JKB 의/JKG 연관/NNG 성/XSN 이/JKS 좀/MAG 더/MAG 높/VA 은/ETM 것/NNB 으로/JKB 분석/NNG 되/XSV 므로/EC 경기/NNG 상환/NNG 판단/NNG 을/JKO 위해서/VV+EC 는/JX 공식/NNG 적/XSN 인/VCP+ETM 실업/NNG 률/XSN 보다/JKB 는/JX 광의/NNG 의/JKG 실업/NNG 률/XSN 을/JKO 이용/NNG 하/XSV 는/ETM 것/NNB 이/JKS 더/MAG 좋/VA 을/ETM 것/NNB 같/VA 다고/EC 설명/NNG 하/XSV 였/EP 음/ETN
['이', '에', '대해', '관련', '부서', '에서', '는', '실업', '률', '과', '경기', '지수', '와', '의', '상관', '관계', '는', '그렇게', '높', '지', '않', '은', '것', '으로', '나타나', '며', '통계청', '에서', '발표', '하', '는', '공식', '적', '인', '실업', '률', '을', '구직', '단념', '자', '를', '포함', '한', '광의의실업', '률', '로', '전환', '할', '경우', '오히려', '경기', '와', '의', '연관', '성', '이', '좀', '더', '높', '은', '것으로분석', '되', '므

In [9]:
tokens = ngram[sentence]
print(tokens)

['이', '에', '대해', '관련', '부서', '에서', '는', '실업', '률', '과', '경기', '지수', '와', '의', '상관', '관계', '는', '그렇게', '높', '지', '않', '은', '것', '으로', '나타나', '며', '통계청', '에서', '발표', '하', '는', '공식', '적', '인', '실업', '률', '을', '구직', '단념', '자', '를', '포함', '한', '광의의실업', '률', '로', '전환', '할', '경우', '오히려', '경기', '와', '의', '연관', '성', '이', '좀', '더', '높', '은', '것으로분석', '되', '므로', '경기', '상환', '판단', '을', '위해서', '는', '공식', '적', '인', '실업', '률', '보다', '는', '광의의실업', '률', '을', '이용', '하', '는', '것', '이', '더', '좋', '을', '것', '같', '다고', '설명', '하', '였', '음']


In [17]:
_ngrams =ngram.find_ngrams(
    ngram.sentences[10:105], strip_pos=False, surface_delim=";", threshold=0.5
)

In [18]:
_ngrams

{}